# Locations of nearby Hot DA white dwarfs

Figure 5.9 from Chapter 5 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

Replotting of of Figure 1 from [Savage & Lehner 2006, ApJS, 162, 134](https://ui.adsabs.harvard.edu/abs/2006ApJS..162..134S) of the locations on the sky of nearby
hot DA white dwarfs used to try to detect OVI absorption in the local ISM.

We have extracted the relevant data from Tables 1 and 3 of Savage & Lehner (2006), and are replotting their
Figure 1 for use in the ISM/IGM book.  Specifically, we are adopting the same Mollweide projection used for
the all-sky maps in the book (the original was an Hammer-Aitoff projection), and omitting the labels on the
points.  The data are in a separate text file associated with this notebook named sl2006_wd.txt

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)
warnings.filterwarnings('ignore',category=RuntimeWarning, append=True)


## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.


In [ ]:
figName = 'Fig5_9' 

# graphic aspect ratio = width/height

aspect = 16.0/9.0 # 16:9 aspect ratio

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# Plot filename

plotFile = f'{figName}.{figFmt}'

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## Savage & Lehner (2006) White Dwarf Data

Data is an ASCII column-oriented file, with the following labeled columns:
 * name - name of the white dwarf
 * lon - galactic longitude in decimal degrees
 * lat - galactic latitude in decimal degrees
 * dpc - distance in parsecs
 * logNa - base-10 logarithm of the column density of OVI in $cm^{-2}$
 * lim - limit flag, Y=OVI upper limit, N=OVI detected

In [ ]:
dataFile = 'sl2006_wd.txt'

data = pd.read_csv(dataFile,sep=r'\s+',comment='#')

lon = np.array(data['lon'])
lat = np.array(data['lat'])
dpc = np.array(data['dpc'])
logNa = np.array(data['logNa'])
lim = np.array(data['lim'])

upperLimits = np.where(lim=='Y')[0]
detections  = np.where(lim=='N')[0]

## All-Sky Distribution Plot

Plot the distribution of objects on the sky in galactic coordinates with a Mollweide projection,
formatting each point as follows:
 * The marker shape is a circle for detections, upside-down triangle for upper limits
 * The marker area is scaled to distance, closer = larger
 * The marker color is scaled to the log10 of the OVI column in the range $\log N_a(O\,VI)=12.4-13.6$
 
We have to mess with the longitude data in order to plot it in the conventional astronomy all-sky
East=left orientation, as the default projection in matplotlib is looking from outside onto the sphere,
not inside looking out.

In [ ]:
# make true if you want a color bar along the bottom showing the column density shading of the markers

plotColorBar = False

# scale marker face color by log OVI column density

minNa = 12.4
maxNa = 13.6

# scale marker by area scaled by distance in reverse (closer = larger)

area = 0.01 * ((200-dpc)**2)

# Make the plot

fig,ax = plt.subplots()

fig.set_dpi(dpi)
fig.set_size_inches(wInches,hInches,forward=True)

# Transform lon->xlon running with conventional East=left for Mollweide projection

xlon = np.remainder(lon+360.0,360.0) 
index = xlon>180.0
xlon[index] -=360.0    
xlon = -xlon           
tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
tick_labels = np.remainder(tick_labels+360,360)

ax=plt.axes(projection='mollweide')
ax.set_xticklabels(tick_labels)
ax.yaxis.set_ticklabels([])    
plt.grid(ls=':',lw=0.3,color='black')

# Convert to radians for matplotlib scatter()

rlon = np.radians(xlon)
rlat = np.radians(lat)

# Make a series of scatter plots of the different symbol types

scb = ax.scatter(rlon[detections],rlat[detections],c=logNa[detections],vmin=minNa,vmax=maxNa,cmap='Greys',
                 marker='o',s=area[detections],edgecolors='black',lw=0.2,clip_on=False,zorder=10)

ax.scatter(rlon[upperLimits],rlat[upperLimits],c=logNa[upperLimits],vmin=minNa,vmax=maxNa,cmap='Greys',
           marker='v',s=area[upperLimits],edgecolors='black',lw=0.2,clip_on=False,zorder=10)

if plotColorBar:
    cb = plt.colorbar(scb,orientation='horizontal',shrink=0.5,pad=0.05)
    cb.set_label(label=r'log$_{10}$(N$_a$(O\,IV))',size=10)
    cb.ax.tick_params(labelsize=10)

# make the file

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')